In [19]:
from qiskit_nature.second_q.drivers import ElectronicStructureDriver, PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
# https://www.webqc.org/molecularformatsconverter.php

In [20]:
# https://docs.python.org/3/tutorial/inputoutput.html
with open("DataBank Folder/menthol_tester.xyz") as f:
    s = f.readlines()
xyz = s[2:]
print(xyz)
def process_xyz(file):
    total = []
    symbols = [i.split()[0] for i in file]
    total = [tuple(i.split()[1:]) for i in file]
    return symbols, total
molecule = MoleculeInfo(
    symbols = process_xyz(xyz)[0],
    coords = process_xyz(xyz)[1],
    multiplicity=1,
    charge=0
)


['C         -1.99200        1.15000        0.00000\n', 'C         -1.99200       -0.39100        0.00000\n', 'C         -0.66100       -1.15700        0.00000\n', 'C          0.66200       -0.39100        0.00000\n', 'C          0.66200        1.15000        0.00000\n', 'C         -0.66100        1.92400        0.00000\n', 'C         -0.66100       -2.69800        0.00000\n', 'C         -1.99600       -3.46500        0.00000\n', 'C          0.66700       -3.46500        0.00000\n', 'C         -0.66100        3.46500        0.00000\n', 'O          1.99600       -1.16700        0.00000']


In [21]:
from qiskit_nature.second_q.problems import ElectronicStructureProblem
driver = PySCFDriver.from_molecule(molecule, basis="sto3g")
problem = driver.run()

In [22]:
from qiskit_nature.second_q.drivers import PySCFDriver
driver = PySCFDriver(
    atom="; ".join([i.replace('\n', ' ') for i in xyz]),
    unit = DistanceUnit.ANGSTROM, 
)
problem = driver.run()

KeyboardInterrupt: 

In [4]:
"; ".join([i.replace('\n', ' ') for i in xyz])

'C         -1.99200        1.15000        0.00000 ; C         -1.99200       -0.39100        0.00000 ; C         -0.66100       -1.15700        0.00000 ; C          0.66200       -0.39100        0.00000 ; C          0.66200        1.15000        0.00000 ; C         -0.66100        1.92400        0.00000 ; C         -0.66100       -2.69800        0.00000 ; C         -1.99600       -3.46500        0.00000 ; C          0.66700       -3.46500        0.00000 ; C         -0.66100        3.46500        0.00000 ; O          1.99600       -1.16700        0.00000'

In [23]:
problem.hamiltonian.electronic_integrals.register_length
problem.hamiltonian

In [6]:
second_qs = problem.second_q_ops()

KeyboardInterrupt: 

In [ ]:
https://github.com/kjwcoo/Qiskit_Hackathon_Korea_2022_Nature/blob/main/9037647705611511190qiskit_nature_challenge.ipynb
https://github.com/qiskit-community/qiskit-application-modules-demo-sessions/blob/main/qiskit-nature/Qiskit_Nature_YouTube_Tutorial.ipynb

In [ ]:
Insights

# Insights
Electronic Structure (Kinetic, Potential, Coulomb + Exchange)
Born Oppenheimer -> Separate e motion from nuclei
Non-rel TI Schro Eq -> Use Variational Principle
Second Quantization (Map to Spin / Qubits)
Hamiltonian -> Creation + Annihilation (Excite + De-excite) -> Occupation Number Vector

Problem -> Solver -> Result
Problem (Driver -> Properties tfed by Transformers ) -> Ferm Op

Op -> Solver (Converter -> Operator + Eigensolver) -> Result

In [39]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
transformer = ActiveSpaceTransformer(
    num_electrons = 4,
    num_spatial_orbitals = 4,
)
#transformer.prepare_active_space(problem.num_particles, problem.num_spatial_orbitals)
#reduced_ham = transformer.transform_hamiltonian(problem.hamiltonian)
red_problem  = transformer.transform(problem)
#red_problem = ElectronicStructureProblem(driver, [transformer])

In [40]:
#new_problem = ElectronicStructureProblem(reduced_ham)
print(red_problem.num_particles)

(2, 2)


In [41]:
#new_problem = ElectronicStructureProblem(reduced_ham)
#print(new_problem.second_q_ops())
reduced_ham = red_problem.second_q_ops()[0]
from qiskit_nature.second_q.mappers import JordanWignerMapper
mapper = JordanWignerMapper()
qubit_ham = mapper.map(reduced_ham)

In [42]:
print(qubit_ham)

SparsePauliOp(['IIIIIIII', 'IIIIIIIZ', 'IIIIYZZY', 'IIIIXZZX', 'IIIIIIZI', 'IIIIIYYI', 'IIIIIXXI', 'IIIIIZII', 'IIIIZIII', 'IIIZIIII', 'YZZYIIII', 'XZZXIIII', 'IIZIIIII', 'IYYIIIII', 'IXXIIIII', 'IZIIIIII', 'ZIIIIIII', 'IIIIIIZZ', 'IIIIIYYZ', 'IIIIIXXZ', 'IIIIIZIZ', 'IIIIZIIZ', 'IIIZIIIZ', 'YZZYIIIZ', 'XZZXIIIZ', 'IIZIIIIZ', 'IYYIIIIZ', 'IXXIIIIZ', 'IZIIIIIZ', 'ZIIIIIIZ', 'IIIIYZIY', 'IIIIXZIX', 'IIIIYYYY', 'IIIIXXYY', 'IIIIYXXY', 'IIIIXYYX', 'IIIIYYXX', 'IIIIXXXX', 'IIYYIIYY', 'IIXXIIYY', 'IIYYIIXX', 'IIXXIIXX', 'IYZYIIYY', 'IXZXIIYY', 'IYZYIIXX', 'IXZXIIXX', 'YZYIIIYY', 'XZXIIIYY', 'YZYIIIXX', 'XZXIIIXX', 'YYIIIIYY', 'XXIIIIYY', 'YYIIIIXX', 'XXIIIIXX', 'IIIIYIZY', 'IIIIXIZX', 'IIYYIYZY', 'IIXXIYZY', 'IIYYIXZX', 'IIXXIXZX', 'IYZYIYZY', 'IXZXIYZY', 'IYZYIXZX', 'IXZXIXZX', 'YZYIIYZY', 'XZXIIYZY', 'YZYIIXZX', 'XZXIIXZX', 'YYIIIYZY', 'XXIIIYZY', 'YYIIIXZX', 'XXIIIXZX', 'IIIZYZZY', 'IIIZXZZX', 'YZZYYZZY', 'XZZXYZZY', 'YZZYXZZX', 'XZZXXZZX', 'IIZIYZZY', 'IIZIXZZX', 'IYYIYZZY', 'IXXIYZZY', '

In [37]:
print(red_problem.num_particles)

(2, 2)


In [43]:
print(red_problem.properties)
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
#from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    red_problem.num_spatial_orbitals,
    red_problem.num_particles,
    mapper,
    initial_state = HartreeFock(
        red_problem.num_spatial_orbitals,
        red_problem.num_particles,
        mapper
    )
)
initial_point = HFInitialPoint()
initial_point.ansatz = ansatz
initial_point.problem = red_problem
estimator = Estimator()
solver = VQE(estimator, ansatz, SLSQP())
solver.initial_point = initial_point.to_numpy_array()

In [44]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
gr = GroundStateEigensolver(mapper, solver)
results = gr.solve(red_problem)

In [46]:
print(results)
with open("MentholResults.txt", "w") as f:
    f.write(results)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -843.245349110765
  - computed part:      -2.368293396647
  - ActiveSpaceTransformer extracted energy part: -840.877055714118
~ Nuclear repulsion energy (Hartree): 396.62171469262
> Total ground state energy (Hartree): -446.623634418145
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-45.03217355  -61.61263057  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-43.739707823134  -60.224392762598  0.00000007901]
    - computed part:      [-3.941595914411  -1.68418257211  0.000000079009]
    - ActiveSpaceTransformer extracted energy part: [-39.798111908724  -58.540210190488  0.000000000001]
  > Dipole moment (a.u.): [-1.292465726866  -1.388237807402  -0.00000007901]  Total: 1.896752979442
                 (debye): [-3.285119890028  -3.528548214772  -0.000000200823]  Total: 4.821064736739
 


TypeError: write() argument must be str, not ElectronicStructureResult